In [1]:
import pyspark
sc = pyspark.SparkContext(master="local[4]",appName="RDDfun")

21/09/20 11:52:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Ford GoBike's trip data

https://www.fordgobike.com/system-data


Each trip is anonymized and includes:
- Trip Duration (seconds)
- Start Time and Date
- End Time and Date
- Start Station ID
- Start Station Name
- Start Station Latitude
- Start Station Longitude
- End Station ID
- End Station Name
- End Station Latitude
- End Station Longitude
- Bike ID
- User Type (Subscriber or Customer – “Subscriber” = Member or “Customer” = Casual)
- Member Year of Birth
- Member Gender


In [2]:
rawData = sc.textFile("2017-fordgobike-tripdata.csv")
rawData.take(2)

['"duration_sec","start_time","end_time","start_station_id","start_station_name","start_station_latitude","start_station_longitude","end_station_id","end_station_name","end_station_latitude","end_station_longitude","bike_id","user_type","member_birth_year","member_gender"',
 '"80110","2017-12-31 16:57:39.6540","2018-01-01 15:12:50.2450","74","Laguna St at Hayes St","37.776434819204745","-122.42624402046204","43","San Francisco Public Library (Grove St at Hyde St)","37.7787677","-122.4159292","96","Customer","1987","Male"']

In [3]:
col_names = rawData.filter(lambda x: x.startswith('"duration'))\
.flatMap(lambda x: x.split(",")).map(lambda x: x[1:-1]).collect()
col_names

['duration_sec',
 'start_time',
 'end_time',
 'start_station_id',
 'start_station_name',
 'start_station_latitude',
 'start_station_longitude',
 'end_station_id',
 'end_station_name',
 'end_station_latitude',
 'end_station_longitude',
 'bike_id',
 'user_type',
 'member_birth_year',
 'member_gender']

[datetime](https://docs.python.org/3/library/datetime.html#module-datetime)

In [4]:
from datetime import datetime, timedelta
dt_format = "%Y-%m-%d %H:%M:%S.%f"

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

> **TODO**: Utwórz RDD `transData` gdzie elementami są listy stringów (odfiltruj element zawierający nazwy)

In [5]:
transData = rawData.filter(lambda x: not x.startswith('"duration')).map(lambda row: [x[1:-1] for x in row.split(",")])
transData.take(1)[0]

['80110',
 '2017-12-31 16:57:39.6540',
 '2018-01-01 15:12:50.2450',
 '74',
 'Laguna St at Hayes St',
 '37.776434819204745',
 '-122.42624402046204',
 '43',
 'San Francisco Public Library (Grove St at Hyde St)',
 '37.7787677',
 '-122.4159292',
 '96',
 'Customer',
 '1987',
 'Male']

> **TODO**: Sprawdź czy, gdzie i w ilu rekordach występują braki danych w RDD `transData` (hint: elementami RDD są listy stringów)

In [6]:
transData.filter(lambda x: "" in x).count()

66541

In [12]:
transData.flatMap(lambda row: [i for i, x in enumerate(row) if x == ""]).distinct().collect()

[13, 14]

> **TODO**: Uwzględniając informacje uzyskane w poprzednim ćwiczeniu utwórz RDD `data` którego elementy będą słownikami. Słowniki te mają zawierać klucze o wartościach następujących typów:
- 'duration_sec' : "int"
- 'start_time' : "datetime"
- 'end_time' : "datetime"
- 'start_station_id' : "int"
- 'start_station_name' : "string"
- 'start_station_latitude' : "float"
- 'start_station_longitude' : "float"
- 'end_station_id' : "int"
- 'end_station_name' : "string"
- 'end_station_latitude' : "float"
- 'end_station_longitude' : "float"
- 'bike_id' : "int"
- 'user_type' : "string"
- 'member_birth_year' : "int"
- 'member_gender' : "string"

> (hint: autor zadania rozwiązał je pisząc funkcję przekształcającą elementy w RDD)

In [13]:
struct = ["int", 
          "datetime", 
          "datetime", 
          "int", 
          "string", 
          "float", 
          "float", 
          "int", 
          "string", 
          "float", 
          "float", 
          "int", 
          "string", 
          "int", 
          "string"]
struct

['int',
 'datetime',
 'datetime',
 'int',
 'string',
 'float',
 'float',
 'int',
 'string',
 'float',
 'float',
 'int',
 'string',
 'int',
 'string']

In [14]:
col_names

['duration_sec',
 'start_time',
 'end_time',
 'start_station_id',
 'start_station_name',
 'start_station_latitude',
 'start_station_longitude',
 'end_station_id',
 'end_station_name',
 'end_station_latitude',
 'end_station_longitude',
 'bike_id',
 'user_type',
 'member_birth_year',
 'member_gender']

In [15]:
def transform_element(RDD_elem, elem_struct, names, dt_fmt):
    out = dict()
    for elem, e_type, name in zip(RDD_elem, elem_struct, names):
        if e_type == "int":
            if elem == "":
                elem = 9999
            else:
                elem = int(elem)
        elif e_type == "datetime":
            elem = datetime.strptime(elem, dt_fmt)
        elif e_type == "float":
            elem = float(elem)
        out[name] = elem
    return out

In [16]:
x = transData.first()

transform_element(x, struct, col_names, dt_format)

{'duration_sec': 80110,
 'start_time': datetime.datetime(2017, 12, 31, 16, 57, 39, 654000),
 'end_time': datetime.datetime(2018, 1, 1, 15, 12, 50, 245000),
 'start_station_id': 74,
 'start_station_name': 'Laguna St at Hayes St',
 'start_station_latitude': 37.776434819204745,
 'start_station_longitude': -122.42624402046204,
 'end_station_id': 43,
 'end_station_name': 'San Francisco Public Library (Grove St at Hyde St)',
 'end_station_latitude': 37.7787677,
 'end_station_longitude': -122.4159292,
 'bike_id': 96,
 'user_type': 'Customer',
 'member_birth_year': 1987,
 'member_gender': 'Male'}

In [17]:
data = transData.map(lambda row: transform_element(row, struct, col_names, dt_format))

In [18]:
data.first()

{'duration_sec': 80110,
 'start_time': datetime.datetime(2017, 12, 31, 16, 57, 39, 654000),
 'end_time': datetime.datetime(2018, 1, 1, 15, 12, 50, 245000),
 'start_station_id': 74,
 'start_station_name': 'Laguna St at Hayes St',
 'start_station_latitude': 37.776434819204745,
 'start_station_longitude': -122.42624402046204,
 'end_station_id': 43,
 'end_station_name': 'San Francisco Public Library (Grove St at Hyde St)',
 'end_station_latitude': 37.7787677,
 'end_station_longitude': -122.4159292,
 'bike_id': 96,
 'user_type': 'Customer',
 'member_birth_year': 1987,
 'member_gender': 'Male'}

> **TODO**: Wykonaj cache() na RDD `data`

In [19]:
data.cache()

PythonRDD[20] at RDD at PythonRDD.scala:53

> **ZADANIE 1**: Zapoznaj się z danymi:
1. Sprawdź rozkład zmiennej "member_gender"
2. Oblicz minimalny, maksymalny i średni wiek osób wypożyczających rowery
3. Oblicz liczbę unikalnych rowerów
4. Oblicz liczbę unikalnych stacji
5. Sprawdź który rower był wypożyczony najdłużej a który najkrócej w ciągu analizowanego okresu (oraz jak długo)
6. Oblicz średni czas pojedynczego wypożyczenia
7. Sprawdź pomiędzy którymi stacjami występował największy ruch
8. Sprawdź o której godzinie w ciągu dnia wypożyczano najwięcej rowerów
9. Sprawdź *średnią liczbę wypożyczeń* dla poszczególnych dni tygodnia

In [24]:
# 1
data.map(lambda x: (x["member_gender"], 1)).reduceByKey(lambda x,y: x+y).collect()

[('', 66462), ('Male', 348318), ('Female', 98621), ('Other', 6299)]

In [28]:
# 2
data.filter(lambda x: x["member_birth_year"] != 9999).map(lambda x: 2021 - x["member_birth_year"]).stats()

(count: 453159, mean: 40.59521271783174, stdev: 10.513475939682797, max: 135.0, min: 22.0)

In [30]:
# 3
data.map(lambda x: x["bike_id"]).distinct().count()

3673

In [32]:
# 4
data.map(lambda x: x["start_station_id"]).union(data.map(lambda x: x["end_station_id"])).distinct().count()

272

In [43]:
# 5
[data.map(lambda x: (x["bike_id"], x["duration_sec"]))\
 .reduceByKey(lambda x,y: x+y).takeOrdered(1, (lambda x: x[1])),
 data.map(lambda x: (x["bike_id"], x["duration_sec"]))\
 .reduceByKey(lambda x,y: x+y).takeOrdered(1, (lambda x: -x[1]))]


[[(722, 70)], [(775, 555010)]]

In [45]:
# 6
data.map(lambda x: x["duration_sec"]).mean()

1099.009520877427

In [46]:
# 7 ***
data.map(lambda x: (str(x["start_station_id"])+"-"+str(x["end_station_id"]),1) \
         if x["start_station_id"] > x["end_station_id"] \
         else (str(x["end_station_id"])+"-"+str(x["start_station_id"]),1))\
.reduceByKey(lambda x,y: x+y).takeOrdered(1, (lambda x: -x[1]))

[('15-6', 5078)]

In [47]:
[data.filter(lambda x: x["start_station_id"] == 15).map(lambda x: x["start_station_name"]).take(1),
data.filter(lambda x: x["start_station_id"] == 6).map(lambda x: x["start_station_name"]).take(1)]

[['San Francisco Ferry Building (Harry Bridges Plaza)'],
 ['The Embarcadero at Sansome St']]

In [52]:
# 8
data.map(lambda x: (x["start_time"].hour,1)).reduceByKey(lambda x,y: x+y).takeOrdered(1, (lambda x: -x[1]))

[(17, 60111)]

In [65]:
# 9
data.map(lambda x: (x["start_time"].date(), 1))\
.reduceByKey(lambda x,y: x+y).map(lambda x: (x[0].isoweekday(),x[1]))\
.aggregateByKey((0.,0.), (lambda acc,x: (acc[0] + x, acc[1] + 1)), \
                (lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1])))\
.mapValues(lambda x: x[0]/x[1]).takeOrdered(7)


[(1, 3131.153846153846),
 (2, 3379.423076923077),
 (3, 3250.074074074074),
 (4, 3157.1481481481483),
 (5, 3006.1111111111113),
 (6, 1884.2222222222222),
 (7, 1681.148148148148)]